# Lab02 - Predicción y Clasificación

In [ ]:
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import pylab as pl
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn import svm
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

%matplotlib inline

## Cargar el set de datos
Vamos a continuar usando el set de datos que limpiamos y transformamos en el [laboratorio pasado](./Lab01py_Preprocesamiento.ipynb) en donde se registran los atributos de varias botellas de vino junto con su calidad.  De esta forma usaremos el mismo dataset a lo largo de todo el flujo del curso como si se tratara de un proyecto real!

In [ ]:
wine_df = pd.read_csv('data/winequality-white_clean.csv')
len(wine_df)

In [ ]:
wine_df.head()

## 2a. Predicción  
Ya hemos visto los conceptos básicos de una regresión: 
- Aprendizaje **supervisado**
- Predecir una variable **contínua**
- Se busca encontrar los **pesos** de las variables para validar cada una de ellas cómo influye en nuestra variable final  

Lo primero que debemos hacer es separar nuestra variable objetivo (*score*) de las variables predictivas (En este caso son todas las demás, excepto *high_quality* que será nuestra variable objetivo en el laboratorio de clasificación)

In [ ]:
y = wine_df['score']
x = wine_df.drop(['score'], axis=1)
x.head()

In [ ]:
print('Población:'+str(len(x)))
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
print('Muestra para Entrenamiento:'+str(len(x_train)))
print('Muestra para Pruebas:'+str(len(x_test)))

### OLS (Ordinary Least Squares)

Llamado así porque busca los coeficientes de la ecuación que minimizen la suma de las distancias verticales entre los datos y el modelo.  Para aplicar este modelo se debe validar que no haya multicolinealidad entre las variables predictivas, pues no la detecta fácilmente.  
![OLS](https://miro.medium.com/max/3268/1*AwC1WRm7jtldUcNMJTWmiA.png)

- **Precisión:** Alta, para variables no correlacionadas (Condiciones para OLS [aquí](https://statisticsbyjim.com/regression/ols-linear-regression-assumptions))
- **Velocidad:** Rápido
- **Explicativo:** Mucho (Peso/Importancia de cada variable)
- **Sensible a cambios:** Poco, OLS crea una estimación generalizada a todos los puntos, un solo valor atípico modificará los pesos de las variables pero no drasticamente, pues seguirá convergiendo a los demás puntos
- **Deteminístico**: Sí

In [ ]:
x.columns

Empecemos con un modelo sencillo: una regresión sobre una sola variable.  Elija una de las columnas predictivas y usémosla como única variable predictiva.  *Ej: chlorides*

In [ ]:
model_ols = smf.ols('score ~ chlorides', data=wine_df)
fit_ols = model_ols.fit()

En el caso de **una** variable predictiva y una variable objetivo, podemos ver gráficamente el comportamiento de nuestro modelo:

In [ ]:
fig, ax = plt.subplots()
plt.scatter(x['chlorides'], y, c='Purple', label='Datos')
y_pred = fit_ols.predict(x['chlorides'])
ax.plot(x['chlorides'], y_pred, c='Blue', label='Modelo')
plt.xlabel('chlorides')
plt.ylabel('score')
plt.title('Primer Modelo de Regresión Lineal Simple');

Con una sola variable es posible que el modelo no se ajuste correctamente a los datos, podemos ver que daría predicciones erradas.  Podemos incluír otras variables al modelo que nos permitan darle más flexibilidad a l ecuación

En el parámetro para *smf.ols()* se le indica al modelo que necesitamos una función de la forma:
> Y ~ X1 + X2 + X3 + X4 + ... + Xn  

En donde Y es nuestra variale objetivo y cada Xi son las variables predictivas para las que se debe buscar un peso en la ecuación.

In [ ]:
model_ols = smf.ols('score ~ fixed_acidity + volatile_acidity + citric_acid + residual_sugar +\
              chlorides + free_sulfur_dioxide + total_sulfur_dioxide + density + pH+\
              sulphates + alcohol', data=wine_df)
fit_ols = model_ols.fit()
fit_ols.summary()

La funcion *fit.summary()* nos entrega un montón de información, vamos a analizar las que ma´s nos interesan:  

- Modelo utilizado (OLS), cantidad de registros y de variables usadas (incluyendo intercepto)
- Df Residuals (Flexibilidad del modelo: #Observacionnes - #Variables) y Df modelo (#Variables -Intercepto) *(Un dato que no da el summary de R)*
- R2, en la primera tabla, es el cuadrado del coeficiente de correlación de todas las variables y representa el porcentaje estimado de varianza que puede ser explicado por el modelo.  
- Adj. R2 es un R2 que penaliza coeficientes muy grandes o muy bajos y variables redundantes.  
- La tabla del centro nos muestra los coeficientes encontrados por el modelo para cada variable, junto con el error estándar de cada uno.  Esto significa que nuestro modelo tiene la forma:

$$
y = 153.5215 + 0.1088(fixed acidity) - 1.6731(volatile acidity) + ... + 0.2012(alcohol)
$$

- Los valores t y valores P (*P>t*), para cada variable, indican si esta afecta o no la variable objetivo.  Son valores estadísticos, por lo general se busca que P < 0.05
- Intervalo de confianza de cada variable [0.025 - 0.975]

Por ahora, vamos a trabajar con estos valores para mejorar nuestro modelo.  Si quieres saber en detalle el significado y el impacto de modificar cada uno de estos resultados, puedes leer más [aquí](https://www.datarobot.com/blog/ordinary-least-squares-in-python/)

**Taller**  
Juegue un poco con las variables en la función *smf.ols()* para ver si logra aumentar el R2 del modelo.  Por ejemplo:
- Revise las variables que, según el  [laboratorio pasado](./Lab01py_Preprocesamiento.ipynb) están correlacionadas entre sí y aquellas que están correlacionadas con la variabe objetivo
- Revise las variables que, según el modelo afectan la variable objetivo (*P valores*)  

Una vez encuentre un modelo que considere adecuado, vamos a a guardarlo para futuros laboratorios:

In [ ]:
fit_ols.save('models/model_ols.pkl')

### Modelos Polinomiales  
Modelos como el anterior, en los que se define una *"fórmula"* para que el modelo encuentre los coeficientes, son muy comunes en las funciones de R.  
[scikit-learn](https://scikit-learn.org/dev/index.html), es la librería preferida de los que trabajamos con Machine Learning en **Python**.  No solo para predicción de variables contínuas sino para todo tipo de modelos.  
>Por ejemplo, el ejercicio anterior puede realizarse también con esta librería usando la clase *LinearRegression()*  

Para ello, es necesario dividir el dataset en entrenamiento y prueba y **diferenciar las variables predictivas (x) de la variable objetivo (y).**

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
columnas = ['residual_sugar', 'alcohol', 'citric_acid']

x_train = x_train[columnas]
x_test = x_test[columnas]

In [ ]:
model_poly = PolynomialFeatures(degree=2)
x_train_poly = model_poly.fit_transform(x_train)
x_test_poly = model_poly.fit_transform(x_test)

model_linpoly = LinearRegression()
model_linpoly.fit(x_train_poly, y_train)
model_linpoly.feature_names_ = model_poly.get_feature_names()

model_linpoly.score(x_test_poly, y_test)

En celdas anteriores hemos dividido el conjunto de datos en dos subconjuntos: Entrenamiento y Prueba.  
Ahora, hemos tomado solo tres de ellas (*residual_sugar, alcohol, citric_acid*), para el ejemplo y creado y entrenado (*fit*) un modelo polinomial usando únicamente el set de entrenamiento, y hemos calculado el $R2$ (*score*) usando el set de pruebas: **23%**, el modelo aún puede mejorar, esto lo haremos más adelante.  
Pero cuál es este modelo/ecuación qué hemos creado, asi como lo vimos en la sección pasada?

In [ ]:
print(model_linpoly.coef_)
print('Cantidad de coeficientes: ' + str(len(model_linpoly.coef_)))

Parecen ser muchos? Porqué? Cómo saber a qué variable corresponde cada uno?

In [ ]:
' + '.join(['('+str(c)+')*'+f for c,f in 
          zip(model_linpoly.coef_, model_linpoly.feature_names_)])

En este formato $X0, X1 y X2$ corresponden a nuestras columnas elegidas para el ejemplo: *residual_sugar, alcohol, citric_acid*  

Pero no es necesario realizar este calculo cada vez que queramos predecir un dato nuevo. El modelo nos entrega predicciones de un nuevo set de datos usando la función *predict*

In [ ]:
print(model_linpoly.predict(x_test_poly))

**Taller**  
Juegue un poco con las variables a incluir o no en el modelo, así como el grado de la regresión polinomial para ver si logra aumentar el score del modelo.

Una vez encuentre un modelo que considere adecuado, vamos a a guardarlo para futuros laboratorios:

In [ ]:
with open("models/model_poly.pkl", "wb") as f:
    pickle.dump(model_linpoly, f)

## 2b. Clasificación  
Ya hemos visto porqué una regresión lineal es diferente a una clasificación, tanto en el concepto como matemáticamente: 
- Predecir una variable **categórica**
- Se busca encontrar los patrones o similitudes entre cada *clase* para poder clasificar después un registro nuevo  

Sin embargo, al igual que en la Predicción, lo primero que debemos hacer es separar nuestra variable objetivo (*high_quality*) de las variables predictivas (En este caso son todas las demás, excepto *score* que fue nuestra variable objetivo en el laboratorio de predicción)

In [ ]:
wine_df = pd.read_csv('data/winequality-red_clean.csv')
len(wine_df)

In [ ]:
y = wine_df['high_quality']
x = wine_df.drop(['high_quality'], axis=1)
x.head()

### Regresion Logística

Similar a una regresión lineal, busca definir el comportamiento de los datos a través de una **ecuación**, con la diferencia que en este caso estamos buscando una variable categórica (1/0 o SI/NO), por lo que no nos serviría una ecuación que retorne un valor entre $(-\infty, \infty)$.  
Sin embago, al ser una ecuación, asume que los datos son linealmente separables:

![Logit](https://qph.fs.quoracdn.net/main-qimg-4ddda954955cabf0fb59364cc37232b7.webp)

- **Precisión:** Alta, para variables no correlacionadas y modelos linealmente separables
- **Velocidad:** Rápido
- **Explicativo:** Mucho (Peso/Importancia de cada variable)
- **Sensible a cambios:** Mucho, asume que los datos son separables linealmente
- **Deteminístico**: No, probabilístico

Como el hecho de que sean datos linealmente separables es **muy** importante, vamos a crear un modelo con solo **dos** variables precitivas para poder ver la separación gráficamente:

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
columnas = ['pH', 'alcohol']
x_train = x_train[columnas]
x_test = x_test[columnas]

In [ ]:
plt.scatter(x[columnas[0]], x[columnas[1]], c=y, s=20)
plt.xlabel(columnas[0])
plt.ylabel(columnas[1])
plt.title('high_quality');

En este caso, nuestra variable predictiva *high_quality*, representada por el color de los puntos, parece ser linealmente separable por la variable *alcohol*, pero no tanto por la variable *pH*.  

Cómo afecta esto al modelo?

In [ ]:
model_log = LogisticRegression(solver='liblinear')
model_log.fit(x_train, y_train)
print('Error: {}'.format(1 - model_log.score(x_test, y_test)))

Nuevamente hemos entrenado el modelo con los datos de entrenamiento y calculamos el error con datos de entrenamiento, aunque no parece un score tan malo, veamos cómo se comporta el modelo:

In [ ]:
model_log.predict_proba(x_test)

In [ ]:
y_pred = model_log.predict(x_test)
print('Ground Truth: '+str(np.array(y_test)[:20]))
print('Prediciones:  '+str(y_pred[:20]))

**Taller**  
Juegue un poco con las variables a incluir en el modelo para ver si logra aumentar el score del modelo y sus predicciones.  Algunas sugerencias:
- Valide qué variables permiten la separación lineal de los datos
- Puede incluir mas de dos variables para entrenar el modelo, es lo más recomendable, pero recuerde que no será posible visualizar la separación gráficamente
- Intente con el parámetro *solver* de la función LogisticRegression(), éste permite generar otro tipo de líneas que pueden mejorar el modelo (Los posibles valores para el parámetro solver son: *'liblinear', 'newton-cg', 'lbfgs', 'sag' y 'saga'*.  Para saber en detalle qué tipo de modelo genera cada uno puede consultar [aqui](https://medium.com/@venali/conventional-guide-to-supervised-learning-with-scikit-learn-logistic-regression-generalized-e9783c414588))

Una vez encuentre un modelo que considere adecuado, vamos a a guardarlo para futuros laboratorios:

In [ ]:
with open("models/model_log.pkl", "wb") as f:
    pickle.dump(model_log, f)

### Vecinos más cercanos (kNN)

Es un método de clasificación para problemas **no lineales**, y tal vez con una metodología/matemática más sencilla.
Consiste en ver los datos como puntos en un plano (de N dimensiones cuántas variables incluya el modelo) y clasificar cualquier "punto nuevo" en la misma clase a la que pertenescan los *k* puntos más cercanos (*k* puede ser definido por nosotros mismos)

<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTIr4A7USCWANPP8HDtHIOmN2HOY_CK1PhL2Q&usqp=CAU" alt="kNN" width="400"/>

- **Precisión:** Depende intensamente del valor de k.  muy eficiente para datasets sin ruido
- **Velocidad:** Depende de dimensiones (*Realmente no construye ningún modelo*)
- **Explicativo:** Mucho (Es una ecuación sencilla)
- **Sensible a cambios:** Mucho a atributos irrelevantes y la complejidad del modelo
- **Deteminístico**: Sí

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
model_knn = KNeighborsClassifier(n_neighbors=18)
model_knn.fit(x_train, y_train)
print('Error: {}'.format(1 - model_knn.score(x_test, y_test)))

En este caso, se eligió el número de vecinos (k) aleatoriamente, y con razón no obtuvimos un score bueno. Pero cuidado:
- Si k es muy pequeño, tendiendo a 1, el modelo va a decidir por la clase del ejemplo más cercano. Esto hace el modelo muy sensible al ruido o a pequeños cambios.
- Si k es muy grande, tendiendo a N, el modelo no podrá gneralizar.  Nunca analizará patrones, reglas, ni similitudes sino que decidirá siempre *lo que diga la mayoría*  

**Taller**  
Juegue un poco con el valor de k (*n_neighbors* en la función *KNeighborsClassifier*) para mejorar el score del modelo.  
También puede intentar eliminando aquellas variables que generen ruido o atípicos sobre los datos.

Una vez encuentre un modelo que considere adecuado, vamos a a guardarlo para futuros laboratorios:

In [ ]:
with open("models/model_knn.pkl", "wb") as f:
    pickle.dump(model_knn, f)

### Máquinas de Soporte Vectorial (SVM)

Cuando hablamos de Regresión logística, buscamos **una línea** que divida los datos de la mejor manera. Pero, qué pasa cuando hay más de una línea que cumple con esta condición? Cómo elegir la mejor?  
Las maquinas de soporte vectorial buscan, por el contrario, un *hiperplano*: un conjunto de líneas que maximicen la distancia/margen entre los puntos más cercanos de cada clase.  A estos puntos más cercanos se les conoce como *vectores de soporte*

<img src="https://d2h0cx97tjks2p.cloudfront.net/blogs/wp-content/uploads/sites/2/2017/08/how-svm-works.png" alt="SVM" width="400"/>

- **Precisión:** Alta si se configura correctamente
- **Velocidad:** Lento para una gran cantidad de datos
- **Explicativo:** No
- **Sensible a cambios:** Si, sobretodo a ejemplos mal etiquetados
- **Deteminístico**: Sí

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
model_svm = svm.SVC(probability= True,
                    kernel='rbf')
model_svm.fit(x_train, y_train)
print('Error: {}'.format(1 - model_svm.score(x_test, y_test)))

Aún tenemos un error un poco alto.  Sin embargo, una de las ventajas de SVM, a pesar de ser un algoritmo lento y sensible al ruido, es su versatilidad para manejar problemas no lineales.  Esto lo logra por medio del parámetro **kernell** que permite definir "la forma" del/los hiperplano/s que va a generar:
![Kernell](https://i.imgur.com/HKTLn35.png)

**Taller**  
Juegue un poco con las columas a incluir en el modelo, así como con el **kernell** de la función *svm.SVC()*.  Los posibles valores de este parámetro son:
- 'linear'
- 'poly' (Polinomial, en este caso tambien se puede configurar el parámetro *degree*)
- 'rbf'
- 'sigmoid'

Puede que enuentre varios modelos que le parezcan convenientes.  Se sugiere guardar cada uno de la siguiente manera (cambiando el nombre del archivo):

In [ ]:
with open("models/model_svm.pkl", "wb") as f:
    pickle.dump(model_svm, f)

### Árboles de Decisión

Es el tipo de algoritmo más fácil de entender, pues se basa en reglas condicionales muy similares al lenguaje humano (*Si... entonces...*).  El entrenamiento consiste en encontrar las reglas y condiciones que cubran la mayor cantidad de casos.  
El modelo gráficamente puede ser representado mediante un árbol en el que los **nodos intermedios** son decisiones basadas en alguno de los atributos predictivos, y las **hojas** son la clase objetivo.

<img src="https://upload.wikimedia.org/wikipedia/commons/f/f3/CART_tree_titanic_survivors.png" alt="Decision Tree" width="400"/>

- **Precisión:** Alta tendencia al sobreajuste (memorizar datos)
- **Velocidad:** Rápido. La validación de reglas son condicionales
- **Explicativo:** Sí
- **Sensible a cambios:** Si, pequeños cambios en los datos puede resultar en árboles completamente diferentes.
- **Deteminístico**: No

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
model_tree= DecisionTreeClassifier(max_depth=3)
model_tree.fit(x_train, y_train)
print('Error: {}'.format(1 - model_tree.score(x_test, y_test)))

En cada uno de los nodos, el algoritmo valida qué variable y qué valor de esa variable produce un nivel mayor de "pureza" entre las hojas (*Entropía/gini index*).  Si este nivel no es suficiente para dar una buena decisión, se hace una nueva partición en cada una de las hojas, validando nuevamente todas las variables predictivas.  
Por esto mismo, los árboles de decisión tienen varios parámetros que configurar (Nivel mínimo de entropía, profundidad máxima del árbol, mínima cantidad de muestras para las hojas, etc.)  
Otra de las ventajas de los árboles es que son **tan explicativos**, que pueden ser graficados para validar el proceso de toma de cada decisión en el proceso de predicción:

In [ ]:
plt.figure(figsize=(20,10))
tree.plot_tree(model_tree, 
               feature_names=x.columns,  
               class_names=['BAJA', 'ALTA'],
               filled=True);

**Taller**  
Juegue un poco con las columas a incluir en el modelo, así como con los parámetros de la función *DecisionTreeClassifier()*, como por ejemplo: *max_depth, min_samples_split, min_samples_leaf, max_leaf_nodes, ...*

Una vez encuentre un modelo que considere adecuado, vamos a a guardarlo para futuros laboratorios:

In [ ]:
with open("models/model_tree.pkl", "wb") as f:
    pickle.dump(model_tree, f)

### Ensambles de Modelos
Cuando un modelo es **no determinístico**, pero tiende al **sobreajuste**, es decir, aprende de memoria las reglas y patrones del set de entrenamiento, hasta el punto de no predecir correctamente sobre el test de entrenamiento, esto puede solucionarse haciendo un ensamble o conjunto de modelos.  Esto es, construir varios modelos "débiles" y crear uno "robusto", tomando la decisión final dependiendo de:
- "Democracia", el resultado es lo que diga la mayoría de modelos, realizando cada uno **bajo un set de entrenamiento diferente** (*BaggingClassifier*)
- "Aleatorio", el resultado es lo que diga la mayoría de árboles de decisión, realizando cada uno con un set aleatorio de  variables (*RandomForestClassifier*)

<img src="https://ars.els-cdn.com/content/image/3-s2.0-B9780128177365000090-f09-17-9780128177365.jpg" alt="Rnd Forest" width="500"/>

- **Precisión:** Alta, varios modelos eliminan el sobreajuste
- **Velocidad:** Lento, por velocidad se sacrifica precisión
- **Explicativo:** Sí, aunque no entrega el modelo, entre importancia de variables
- **Sensible a cambios:** No mucho, un dato atípico peude modificar un árbol, pero no todos
- **Deteminístico**: No

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
model_forest= RandomForestClassifier(n_estimators=20, max_depth=5)
model_forest.fit(x_train, y_train)
print('Error: {}'.format(1 - model_forest.score(x_test, y_test)))

Note que, si la anterior celda se ejecuta varias veces, cada vez da un valor diferente.  Esto significa que el modelo es **no determinístico**.  Aunque el error cambie con solo ejecutarlo, es mejor configurar algunos parámetros para indicarle al modelo por dónde está la mejor solución.  
El concenso realizado entre los modelos permite al algoritmo, a pesar de ser no determinístico, darse una idea de la importancia de cada variable en la toma de la decisión:

In [ ]:
for col, imp in zip(x.columns, model_forest.feature_importances_) :
    print('{}: {:.2f}'.format(col, imp))

**Taller**  
Juegue un poco con las columnas a incluir en el modelo, en este caso revise la importancia de variables.  
Puede también jugar con los parámetros de la función *RandomForestClassifier()*, que son los mismos del Árbol de Decisión con la adición de *n_estimators* (Cantidad de modelos/árboles a generar)

Una vez encuentre un modelo que considere adecuado, vamos a a guardarlo para futuros laboratorios:

In [ ]:
with open("models/model_forest.pkl", "wb") as f:
    pickle.dump(model_forest, f)